# LightFM model for influence marketing

Description

### Model theoretical explanation
This model is based in ...

### 1. Import Libraries

In [275]:
# Install all the libraries in requirements.txt
import sys
import os

import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scrapbook as sb
import requests
import io

import lightfm
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm import cross_validation

# Import LightFM's evaluation metrics
from lightfm.evaluation import precision_at_k as lightfm_prec_at_k
from lightfm.evaluation import recall_at_k as lightfm_recall_at_k

# Import repo's evaluation metrics
from recommenders.evaluation.python_evaluation import precision_at_k, recall_at_k

from recommenders.utils.timer import Timer
from recommenders.datasets import movielens
from recommenders.models.lightfm.lightfm_utils import (
    track_model_metrics, prepare_test_df, prepare_all_predictions,
    compare_metric, similar_users, similar_items)

print("System version: {}".format(sys.version))
print("LightFM version: {}".format(lightfm.__version__))

System version: 3.8.7 (tags/v3.8.7:6503f05, Dec 21 2020, 17:59:51) [MSC v.1928 64 bit (AMD64)]
LightFM version: 1.16


### 2. Defining Variables

In [276]:
# Select MovieLens data size
MOVIELENS_DATA_SIZE = '100k'

# default number of recommendations
K = 10
# percentage of data used for testing
TEST_PERCENTAGE = 0.25
# model learning rate
LEARNING_RATE = 0.25
# no of latent factors
NO_COMPONENTS = 20
# no of epochs to fit model
NO_EPOCHS = 20
# no of threads to fit model
NO_THREADS = 32
# regularisation for both user and item features
ITEM_ALPHA = 1e-6
USER_ALPHA = 1e-6

# seed for pseudonumber generations
SEED = 42

### 3. Retrive Data

In [277]:
df_co = pd.read_csv('df_co.csv')
df_co

,CompanyID,Name_co,Category_co,Hashtags_co,Country_co,Followers_co
0,0,The Little Shop,Cars,#instadaily#food#followback#instatravel,Brazil,195608
1,1,The Cozy Kitchen,Sports with a ball,#instalike#fashion#holidayseason#skiing#loveit,Egypt,415795
2,2,The Roost,Art,#blessings#foodlife#endurance#fitnessmotivatio...,Turkey,116164
3,3,The Wooden Spoon,Computers,#foodislifee#familytime,Iran,524384
4,4,Sunflower Fields,Management,#health#trendy,Germany,307127
...,...,...,...,...,...,...
95,95,Copper Kettle,Undefined,#baseball#friendshipgoals#skiing,Thailand,354362
96,96,The Crusty Baguette,Food,#instamood#education#beautifuldestinations,China,74286
97,97,The Plaid Pail,Luxury,#goodmorning#cricket#transportation#media#makeup,Nigeria,108265
98,98,The Sugar House,Business,#muscle#lifestyle#instatravel#instadaily,Germany,82077


In [278]:
df_influ = pd.read_csv('df_inf.csv')
df_influ

,AccountID,Account,Link,Followers,Audience Country,Authentic engagement,Engagement avg,Category1,Hashtags,Cost Story,Cost Post
0,1,cristiano,https://www.instagram.com/cristiano/,400100000.0,India,7800000.0,9500000.0,Sports with a ball,#football#entrepreneur#foodstylist#love#travel...,468000.0,1092000.0
1,2,kyliejenner,https://www.instagram.com/kyliejenner/,308800000.0,United States,6200000.0,10100000.0,Fashion,#blessed#fashionstyle#foodexperience#quotes#fo...,372000.0,868000.0
2,3,leomessi,https://www.instagram.com/leomessi/,306300000.0,Argentina,4800000.0,6500000.0,Sports with a ball,#marathon#foodlover#fashionblogger#motivation#...,288000.0,672000.0
3,4,kendalljenner,https://www.instagram.com/kendalljenner/,217800000.0,United States,3400000.0,5400000.0,Modeling,#food#fitlife#fitnessmodel#vacation#familytime...,204000.0,476000.0
4,5,selenagomez,https://www.instagram.com/selenagomez/,295800000.0,United States,2700000.0,3600000.0,Music,#blackandwhite#fashionblogger#fashion#holidays...,162000.0,378000.0
...,...,...,...,...,...,...,...,...,...,...,...
995,996,senoritasaeva,https://www.instagram.com/senoritasaeva/,7700000.0,Russia,246600.0,318200.0,Lifestyle,#sunset#likeforlike#instagood,14796.0,34524.0
996,997,manuelneuer,https://www.instagram.com/manuelneuer/,11500000.0,Germany,146500.0,210200.0,Sports with a ball,#summer2022#cricket#sundayfunday#beautifulday#...,8790.0,20510.0
997,998,sahilkhan,https://www.instagram.com/sahilkhan/,10100000.0,India,176500.0,239800.0,Fitness,#fitlife#tbt#fashion#fashionable#smilemore#str...,10590.0,24710.0
998,999,mohanshakti,https://www.instagram.com/mohanshakti/,13700000.0,India,146400.0,175500.0,Art,#foodlife#colorful#fit#lifeisgood#telecom#fami...,8784.0,20496.0


### 4. Prepare Data

In [279]:
df2_co = pd.concat([df_co]*1000, ignore_index=True)

In [281]:
df2_influ = pd.DataFrame(np.repeat(df_influ.values, 100, axis=0), columns=["AccountID", "Account", "Link", "Followers", "Audience Country", "Authentic engagement", "Engagement avg", "Category1", "Hashtags", "Cost Story", "Cost Post"])


In [282]:
df = pd.concat([df2_influ, df2_co], axis=1)

In [283]:
def Convert(string):
    li = list(string.split("#"))
    return li

In [284]:
df['Hashtags'] = df['Hashtags'].astype(str)

In [285]:
h = []
h_co = []

for i in range(len(df['Hashtags'])):
    h.append(Convert(df['Hashtags'][i]))
    h_co.append(Convert(df['Hashtags_co'][i]))

df['Hashtags'] = h
df['Hashtags_co'] = h_co

In [286]:
for i in range(len(df['Hashtags'])):
    df['Hashtags'][i].pop(0)
    df['Hashtags_co'][i].pop(0)

In [287]:
df

,AccountID,Account,Link,Followers,Audience Country,Authentic engagement,Engagement avg,Category1,Hashtags,Cost Story,Cost Post,CompanyID,Name_co,Category_co,Hashtags_co,Country_co,Followers_co
0,1,cristiano,https://www.instagram.com/cristiano/,400100000.0,India,7800000.0,9500000.0,Sports with a ball,"[football, entrepreneur, foodstylist, love, tr...",468000.0,1092000.0,0,The Little Shop,Cars,"[instadaily, food, followback, instatravel]",Brazil,195608
1,1,cristiano,https://www.instagram.com/cristiano/,400100000.0,India,7800000.0,9500000.0,Sports with a ball,"[football, entrepreneur, foodstylist, love, tr...",468000.0,1092000.0,1,The Cozy Kitchen,Sports with a ball,"[instalike, fashion, holidayseason, skiing, lo...",Egypt,415795
2,1,cristiano,https://www.instagram.com/cristiano/,400100000.0,India,7800000.0,9500000.0,Sports with a ball,"[football, entrepreneur, foodstylist, love, tr...",468000.0,1092000.0,2,The Roost,Art,"[blessings, foodlife, endurance, fitnessmotiva...",Turkey,116164
3,1,cristiano,https://www.instagram.com/cristiano/,400100000.0,India,7800000.0,9500000.0,Sports with a ball,"[football, entrepreneur, foodstylist, love, tr...",468000.0,1092000.0,3,The Wooden Spoon,Computers,"[foodislifee, familytime]",Iran,524384
4,1,cristiano,https://www.instagram.com/cristiano/,400100000.0,India,7800000.0,9500000.0,Sports with a ball,"[football, entrepreneur, foodstylist, love, tr...",468000.0,1092000.0,4,Sunflower Fields,Management,"[health, trendy]",Germany,307127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1000,eduincaz,https://www.instagram.com/eduincaz/,6200000.0,Mexico,305600.0,391900.0,Lifestyle,[photo],18336.0,42784.0,95,Copper Kettle,Undefined,"[baseball, friendshipgoals, skiing]",Thailand,354362
99996,1000,eduincaz,https://www.instagram.com/eduincaz/,6200000.0,Mexico,305600.0,391900.0,Lifestyle,[photo],18336.0,42784.0,96,The Crusty Baguette,Food,"[instamood, education, beautifuldestinations]",China,74286
99997,1000,eduincaz,https://www.instagram.com/eduincaz/,6200000.0,Mexico,305600.0,391900.0,Lifestyle,[photo],18336.0,42784.0,97,The Plaid Pail,Luxury,"[goodmorning, cricket, transportation, media, ...",Nigeria,108265
99998,1000,eduincaz,https://www.instagram.com/eduincaz/,6200000.0,Mexico,305600.0,391900.0,Lifestyle,[photo],18336.0,42784.0,98,The Sugar House,Business,"[muscle, lifestyle, instatravel, instadaily]",Germany,82077


In [288]:
num_coincidente = df.apply(lambda row: len(set(row['Hashtags']).intersection(set(row['Hashtags_co']))), axis=1)
points_cat = np.where((df['Category1'] == df['Category_co']), 10, 0)
points_country = np.where((df['Audience Country'] == df['Country_co']), 5, 0)
points_eng = df['Authentic engagement']/df['Followers']
df['num_coincidentes'] = num_coincidente
df['points_eng'] = points_eng
df['points_cat'] = points_cat
df['points_country'] = points_country
df['Puntos'] = df['points_cat'] + df['points_country'] + df["num_coincidentes"] + df['points_eng']

df = df.drop(['points_cat', 'points_country', 'num_coincidentes', 'Engagement avg', 'points_eng'], axis=1)


In [289]:
df.dtypes

AccountID               object
Account                 object
Link                    object
Followers               object
Audience Country        object
Authentic engagement    object
Category1               object
Hashtags                object
Cost Story              object
Cost Post               object
CompanyID                int64
Name_co                 object
Category_co             object
Hashtags_co             object
Country_co              object
Followers_co             int64
Puntos                  object
dtype: object

In [290]:
df['Followers'] = df['Followers'].astype(int)
df['Authentic engagement'] = df['Authentic engagement'].astype(float)
df['Cost Story'] = df['Cost Story'].astype(float)
df['Followers_co'] = df['Followers_co'].astype(int)
df['Puntos'] = df['Puntos'].astype(float)
df.dtypes

AccountID                object
Account                  object
Link                     object
Followers                 int32
Audience Country         object
Authentic engagement    float64
Category1                object
Hashtags                 object
Cost Story              float64
Cost Post                object
CompanyID                 int64
Name_co                  object
Category_co              object
Hashtags_co              object
Country_co               object
Followers_co              int32
Puntos                  float64
dtype: object

In [291]:
df

,AccountID,Account,Link,Followers,Audience Country,Authentic engagement,Category1,Hashtags,Cost Story,Cost Post,CompanyID,Name_co,Category_co,Hashtags_co,Country_co,Followers_co,Puntos
0,1,cristiano,https://www.instagram.com/cristiano/,400100000,India,7800000.0,Sports with a ball,"[football, entrepreneur, foodstylist, love, tr...",468000.0,1092000.0,0,The Little Shop,Cars,"[instadaily, food, followback, instatravel]",Brazil,195608,0.019495
1,1,cristiano,https://www.instagram.com/cristiano/,400100000,India,7800000.0,Sports with a ball,"[football, entrepreneur, foodstylist, love, tr...",468000.0,1092000.0,1,The Cozy Kitchen,Sports with a ball,"[instalike, fashion, holidayseason, skiing, lo...",Egypt,415795,10.019495
2,1,cristiano,https://www.instagram.com/cristiano/,400100000,India,7800000.0,Sports with a ball,"[football, entrepreneur, foodstylist, love, tr...",468000.0,1092000.0,2,The Roost,Art,"[blessings, foodlife, endurance, fitnessmotiva...",Turkey,116164,0.019495
3,1,cristiano,https://www.instagram.com/cristiano/,400100000,India,7800000.0,Sports with a ball,"[football, entrepreneur, foodstylist, love, tr...",468000.0,1092000.0,3,The Wooden Spoon,Computers,"[foodislifee, familytime]",Iran,524384,0.019495
4,1,cristiano,https://www.instagram.com/cristiano/,400100000,India,7800000.0,Sports with a ball,"[football, entrepreneur, foodstylist, love, tr...",468000.0,1092000.0,4,Sunflower Fields,Management,"[health, trendy]",Germany,307127,0.019495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1000,eduincaz,https://www.instagram.com/eduincaz/,6200000,Mexico,305600.0,Lifestyle,[photo],18336.0,42784.0,95,Copper Kettle,Undefined,"[baseball, friendshipgoals, skiing]",Thailand,354362,0.049290
99996,1000,eduincaz,https://www.instagram.com/eduincaz/,6200000,Mexico,305600.0,Lifestyle,[photo],18336.0,42784.0,96,The Crusty Baguette,Food,"[instamood, education, beautifuldestinations]",China,74286,0.049290
99997,1000,eduincaz,https://www.instagram.com/eduincaz/,6200000,Mexico,305600.0,Lifestyle,[photo],18336.0,42784.0,97,The Plaid Pail,Luxury,"[goodmorning, cricket, transportation, media, ...",Nigeria,108265,0.049290
99998,1000,eduincaz,https://www.instagram.com/eduincaz/,6200000,Mexico,305600.0,Lifestyle,[photo],18336.0,42784.0,98,The Sugar House,Business,"[muscle, lifestyle, instatravel, instadaily]",Germany,82077,0.049290


Before fitting the LightFM model, we need to create an instance of Dataset which holds the interaction matrix.

In [295]:
dataset = Dataset()

The fit method creates the user(companies)/item(influencers) mappings.

In [297]:
dataset.fit(users=df['CompanyID'], 
            items=df['AccountID'])

# quick check to determine the number of unique users and items in the data
num_users, num_topics = dataset.interactions_shape()
print(f'Num users: {num_users}, num_topics: {num_topics}.')

Num users: 100, num_topics: 1000.


Next is to build the interaction matrix. The build_interactions method returns 2 COO sparse matrices, namely the interactions and weights matrices.

In [298]:
(interactions, weights) = dataset.build_interactions(df.iloc[:, [0,10,16]].values)

ValueError: Item id 0 not in item id mapping. Make sure you call the fit method.